In [1]:
import numpy as np
import keras
from keras.layers import Dense, Input, Add
from keras.models import Sequential, Model, load_model

from Source import RM

Using TensorFlow backend.


In [2]:
m = 5
r = 1
length = 2**m

In [3]:
G = RM.G_matrix(length, m, r)

In [4]:
msg = np.random.randint(2, size=(10**7,m+r))
print(msg)

[[1 1 0 1 0 0]
 [1 1 0 1 1 0]
 [1 0 0 0 1 1]
 ...
 [0 1 1 0 0 0]
 [1 0 0 1 0 1]
 [0 1 1 1 0 0]]


In [5]:
send_code, received_code = RM.Encoding(msg, G, 3, length)

In [6]:
test_msg = np.random.randint(2, size=(10**3,m+r))
test_send_code, test_received_code = RM.Encoding(test_msg, G, 3, length)

In [7]:
input_layer = Input(shape=(32,))
line1_Dense1 = Dense(32*1, activation='elu')(input_layer)
line1_Dense2 = Dense(32*1, activation='elu')(line1_Dense1)         
line1_Dense3 = Dense(32*1, activation='elu')(line1_Dense2)
line1_output_layer = Dense(32, activation='tanh')(line1_Dense3)

line2_Dense1 = Dense(32*2, activation='elu')(input_layer)
line2_Dense2 = Dense(32*4, activation='elu')(line2_Dense1)         
line2_Dense3 = Dense(32*2, activation='elu')(line2_Dense2)
line2_output_layer = Dense(32, activation='tanh')(line2_Dense3)

line3_Dense1 = Dense(32*3, activation='elu')(input_layer)
line3_Dense2 = Dense(32*9, activation='elu')(line3_Dense1)         
line3_Dense3 = Dense(32*3, activation='elu')(line3_Dense2)
line3_output_layer = Dense(32, activation='tanh')(line3_Dense3)

line4_Dense1 = Dense(32*4, activation='elu')(input_layer)
line4_Dense2 = Dense(32*16, activation='elu')(line4_Dense1)         
line4_Dense3 = Dense(32*4, activation='elu')(line4_Dense2)
line4_output_layer = Dense(32, activation='tanh')(line4_Dense3)

line5_Dense1 = Dense(32*5, activation='elu')(input_layer)
line5_Dense2 = Dense(32*25, activation='elu')(line5_Dense1)         
line5_Dense3 = Dense(32*5, activation='elu')(line5_Dense2)
line5_output_layer = Dense(32, activation='tanh')(line5_Dense3)

add_layer = Add()([line1_output_layer, line2_output_layer, line3_output_layer, line4_output_layer, line5_output_layer])

output_layer = Dense(32, activation='linear')(add_layer)

model = Model(inputs = input_layer, outputs=output_layer)

model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

In [8]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32)           0                                            
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 32)           1056        input_1[0][0]                    
__________________________________________________________________________________________________
dense_5 (Dense)                 (None, 64)           2112        input_1[0][0]                    
__________________________________________________________________________________________________
dense_9 (Dense)                 (None, 96)           3168        input_1[0][0]                    
__________________________________________________________________________________________________
dense_13 (

In [9]:
model.fit(received_code, send_code, 
          epochs=100,
          batch_size=10000,
          shuffle=True,
          validation_data=(test_received_code, test_send_code))

Train on 10000000 samples, validate on 1000 samples
Epoch 1/100
10000000/10000000 [==============================] - 35s 4us/step - loss: 0.2147 - acc: 0.0655 - val_loss: 0.0826 - val_acc: 0.1450
Epoch 2/100
10000000/10000000 [==============================] - 30s 3us/step - loss: 0.0550 - acc: 0.0668 - val_loss: 0.0461 - val_acc: 0.0440
Epoch 3/100
10000000/10000000 [==============================] - 30s 3us/step - loss: 0.0399 - acc: 0.0686 - val_loss: 0.0390 - val_acc: 0.0520
Epoch 4/100
10000000/10000000 [==============================] - 29s 3us/step - loss: 0.0350 - acc: 0.0690 - val_loss: 0.0377 - val_acc: 0.0380
Epoch 5/100
10000000/10000000 [==============================] - 29s 3us/step - loss: 0.0323 - acc: 0.0689 - val_loss: 0.0332 - val_acc: 0.0320
Epoch 6/100
10000000/10000000 [==============================] - 28s 3us/step - loss: 0.0306 - acc: 0.0686 - val_loss: 0.0328 - val_acc: 0.1300
Epoch 7/100
10000000/10000000 [==============================] - 28s 3us/step - loss

In [10]:
test_msg = np.random.randint(2, size=(10**3,m+r))
test_send_code, test_received_code = RM.Encoding(test_msg, G, 3, length)

denoise_code = model.predict(test_received_code)

In [11]:
denoise_code = np.where(denoise_code > 0, 0, 1)
test_send_code = np.where(test_send_code > 0, 0, 1)
test_received_code = np.where(test_received_code > 0, 0, 1)

In [12]:
print(denoise_code)
print(test_send_code)

[[1 0 1 ... 1 0 1]
 [0 0 1 ... 1 0 0]
 [1 0 0 ... 0 0 1]
 ...
 [0 1 1 ... 1 1 0]
 [1 0 1 ... 0 1 0]
 [0 1 1 ... 1 1 0]]
[[1 0 1 ... 1 0 1]
 [0 0 1 ... 1 0 0]
 [1 0 0 ... 0 0 1]
 ...
 [0 1 1 ... 1 1 0]
 [1 0 1 ... 0 1 0]
 [0 1 1 ... 1 1 0]]


In [13]:
a = ((test_send_code + denoise_code) % 2).sum()
b = ((test_send_code + test_received_code) % 2).sum()

In [14]:
print(a/(test_send_code.shape[0]*test_send_code.shape[1]))
print(b/(test_send_code.shape[0]*test_send_code.shape[1]))

0.0068125
0.19390625


In [15]:
model.save('Model/model_(1,5)_p5.h5')